In [1]:
import face_rec

C:\Users\tanya\OneDrive\Desktop\Face Recognition Attendance System - Copy - Copy\Attendance_System\Notes\attendance_system\lib\site-packages\albumentations\__init__.py:24: UserWarning: A new version of Albumentations is available: 2.0.3 (you have 1.4.21). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: insightface_model\models\buffalo_l\1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: insightface_model\models\buffalo_l\2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: insightface_model\models\buffalo_l\det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: insightface_model\models\buffalo_l\genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: insightface_model\models\buffalo_l\w600k_r50.onnx recognition ['None', 3, 112, 112] 127.5 127.5
set det-size: (640, 640)


In [2]:
import pandas as pd
import numpy as np
import cv2

In [3]:
face_rec.r.ping()

True

## EXTRACTING DATA FROM REDIS DATABASE

In [4]:
# database name : "sharda:register"
db_name = "sharda"

retrieve_dict = face_rec.r.hgetall(db_name)
retrieve_dict

# array of bytes
# retrieve_dict

retrieve_series = pd.Series(retrieve_dict)
retrieve_series = retrieve_series.apply(lambda x: np.frombuffer(x,dtype=np.float64))
index = retrieve_series.index
index = list(map(lambda x : x.decode(), index))
retrieve_series.index = index

# retrieve_series

# converting retrieve series into df
retrieve_df = retrieve_series.to_frame().reset_index()
retrieve_df.columns = ['name_role', 'facial_features']

# retrieve_df

retrieve_df[['Name', 'Role']] = retrieve_df['name_role'].apply(lambda x: x.split('@')).apply(pd.Series)

for se in retrieve_df['facial_features']:
    print(len(se))

# retrieve_df

# for se in retrieve_df['facial_features']:
#     print(len(se))

512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512


# REAL TIME PREDICTION

## CHECKING FOR INDEX

In [5]:
import cv2

print("Checking available cameras...")
for index in range(5):  # Check indexes from 0 to 4
    cap = cv2.VideoCapture(index, cv2.CAP_DSHOW)
    if cap.isOpened():
        print(f"Camera found at index {index}")
        cap.release()
    else:
        print(f"No camera found at index {index}")


Checking available cameras...
Camera found at index 0
Camera found at index 1
No camera found at index 2
No camera found at index 3
No camera found at index 4


## CAMERA TESTING

In [5]:
import cv2
import time

camera_index = 1  # Replace with correct index
cap = cv2.VideoCapture(camera_index, cv2.CAP_DSHOW)

if not cap.isOpened():
    print(f"Camera at index {camera_index} is not accessible.")
    exit()

cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

time.sleep(2)  # Allow camera to initialize
print("Starting camera feed... Press 'q' to exit.")

while True:
    ret, frame = cap.read()

    if not ret or frame is None:
        print(f"Failed to capture frame from camera index {camera_index}.")
        break

    cv2.imshow("Camera Feed", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):  # Press 'q' to quit
        break

cap.release()
cv2.destroyAllWindows()


Starting camera feed... Press 'q' to exit.


In [7]:
# 0 -> default camera, 1 -> external camera 
# ek baar check krne wala fn chla lio jo phle banya tha, vrna bt aati h
import cv2
import time

camera_index = 1
cap = cv2.VideoCapture(camera_index, cv2.CAP_DSHOW)

cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

if not cap.isOpened():
    print(f"Camera at index {camera_index} is not accessible.")
    exit()

time.sleep(2)  # Allow camera to initialize
print("Starting camera feed... Press 'esc' to exit.")

while True:
    ret, frame = cap.read()
    if ret == False:
        break
        
    pred_frame = face_rec.face_prediction(frame, retrieve_df, "facial_features", ['Name', 'Role'], thresh=0.5)

    # isko baad m hatana h bs testing ke liye likha hu
    # cv2.imshow("frame", frame)
    cv2.imshow("prediction frame", pred_frame)

    if cv2.waitKey(1) == 27:
        break

cv2.destroyAllWindows()
cap.release()

Starting camera feed... Press 'esc' to exit.
